Scoping the feasibility for migrant phenotype project

<u>Process</u>
    
    - list of project codes as csv 
    - map from read to snomend ct 
    - map from snomed ct to gdppr

Gotcha's

 - Multiple read_codes point to the same concept

 - Singular read_codes point to different concept

 - many-many relationship

 - Mapping file has columns IS_ASSURED EffectiveDate MapStatus

 - Filter applied on mapping file was:
 
 
     - IS_ASSURED == 1
     - EffectiveDate == most recent date
     - MapStatus == 1

In [1]:
import pandas as pd 
pd.set_option('display.max_columns', None)

Takes a PDF that has been coverted to csv using adobe online converter and formatted, as a minimum below
___________


| read_code |
| --- | 
|9Nm6.00 |

The output should contain something like:
___________


| read_code | read_code_mapfile | map_id | concept_id_mapfile | concept_id_gdppr | CONCEPT_MAP_FLAG | GPDDR_MAP_FLAG |
| --- | --- | --- | --- | --- | --- | --- | 
|9Nm6.00 | 9Nm6.00 | {f9b73a54-2623-11e3-a0b5-00ff3a5bce8f} | 359821000000104 | 359821000000104 | 1 | 1




Creating python file from Jupyter to make version control and portabilty easier

In [2]:
%%writefile readv2_to_snomed_search_gdppr.py

import pandas as pd

class projectDataPreparation():
    """ 
    Project data preparation before feeding into conversion classes 
    Attributes:
        project_df: the project codelist as a pd.DataFrame
        project_codelist_column: the column in pd.DataFrame with codes   
    
    """
    def __init__(self, 
                 project_df: pd.DataFrame, 
                 project_codelist_column: str):
        
        self.project_df = project_df
        self.project_codelist_column = project_codelist_column
    
    @staticmethod
    def get_num_unique_vars(df: pd.DataFrame) -> pd.Series:
        """
        Prints the number of unique codes per column in project_df
        
        Args:
            df: a pd.DataDrame of the supplied df
            
        Returns:
            a pd.Series with the number of unique values in each column of the supplied df
            
        Example:
            projectDataPreparation.get_num_unique_vars(df)
            
        """
        return df.nunique()

    def get_project_codelist(self) -> pd.array:
        """
        Args:
            self.project_df: the project codelist as a pd.DataFrame
            self.project_codelist_column: the column in pd.DataFrame with codes
            
        Return:
            A pd.array with list of unique codes

        example:
            get_project_codelist(project_dataframe, 'read_code' )

        """
        print(f"""The number of unique codes in the supplied dataframe column '{self.project_codelist_column}' is: 
        {self.project_df[self.project_codelist_column].nunique()}""")
        return self.project_df[self.project_codelist_column].unique()
    
    @staticmethod
    def create_read_term_column(df, read_col: str,term_col: str, 
                                new_read_term_col_name: str) -> pd.DataFrame:
        """
        Creates a combined Read Code and Term Code column for matchin on researcher code list if necessary.
        Args:
            read_col: read codes column
            term_col: term codes column
            new_read_term_col_name: user defined name for the combined read and term columns
            
        Return:
            A pd.DataFrame with derrived read term column

        example:
            projectDataPreparation.create_read_term_column(read_snomed_map, 'ReadCode', 'TermCode','read_term_map')
        """
        
        if not isinstance(df[read_col][0], str) and not isinstance(df[term_col][0], str):
            print(f'Converting {read_col} and {term_col} to strings')
            df[term_col] = df[term_col].astype('str')
            df[read_col] = df[read_col].astype('str')

        else:
            assert isinstance(df[read_col][0], str),f'Data Type for {read_col} is type {type(df[read_col][0])} not {str} '
            assert isinstance(df[term_col][0], str),f'Data Type for {term_col} is type {type(df[term_col][0])} not {str} '

            df[new_read_term_col_name] = df[[read_col, term_col]].apply("".join, axis=1)

            return df
    
class conversionChecks:
    
    @staticmethod
    def get_codes_not_mapped(input_df, 
                             input_df_column, 
                             mapped_df, 
                             mapped_df_column,
                             map_flag_col):
        
#         lost_codes = input_df[~input_df[input_df_column].isin(mapped_df[mapped_df_column].unique())]
        lost_codes = mapped_df.loc[mapped_df[map_flag_col] == 0]
        
        mapped_df =  mapped_df.loc[mapped_df[map_flag_col] == 1]
        
        
        print(f'''
        
        Successfully mapped {mapped_df[mapped_df_column].nunique()} codes from mapped_file of 
         {input_df[input_df_column].nunique()} codes found in the input_df 
        
        ''')

        if len(lost_codes) > 0:
            print(f'{len(lost_codes)} not mapped.')
            print(f'''Printing codes that did not map:
                    {lost_codes}''')
        else:
            None                                                 
        
class readv2ToSnomedConversion(projectDataPreparation):
    def __init__(self, 
                 project_df: pd.DataFrame, 
                 project_codelist_column: str, 
                 mapping_file: pd.DataFrame,
                 mapping_file_codelist_column: str):
        
        self.project_df = project_df
        self.project_codelist_column = project_codelist_column
        self.mapping_file = mapping_file
        self.mapping_file_codelist_column = mapping_file_codelist_column
        super().__init__(project_df, project_codelist_column)
    
    
    def map_readV2_to_snomed(self):
        read_snomed_mapped = self.project_df.merge(
        self.mapping_file, 
            left_on=self.project_codelist_column, 
            right_on=self.mapping_file_codelist_column, 
            how='left', 
            suffixes = ('_project_df_file', '_read_snomed_map_file'))
        
        read_snomed_mapped['CONCEPT_MAP_FLAG'] = (read_snomed_mapped['MapId']
                                                   .where(read_snomed_mapped['MapId']
                                                          .isnull(), 1)
                                                   .fillna(0)
                                                   .astype(int))
        
        return read_snomed_mapped
    
            
class snomedToGdpprLookUp():
    def __init__(self, 
                 mapped_snomed_project_df: pd.DataFrame, 
                 mapped_snomed_project_codelist_column: str, 
                 mapping_file: pd.DataFrame,
                 mapping_file_gdppr_codelist_column: str ):
        
        self.mapped_snomed_project_df = mapped_snomed_project_df
        self.mapped_snomed_project_codelist_column = mapped_snomed_project_codelist_column
        self.mapping_file = mapping_file
        self.mapping_file_gdppr_codelist_column = mapping_file_gdppr_codelist_column
        
        
    def map_snomed_to_gdppr(self):
        
        map_snomed_to_gdppr_df = self.mapped_snomed_project_df.merge(
            self.mapping_file,
            left_on=self.mapping_file_gdppr_codelist_column,
            right_on=self.mapped_snomed_project_codelist_column,
            suffixes = ['_project_codelist','_gdppr'], how='left', indicator=True)
    
        
        map_snomed_to_gdppr_df['GDPPR_MAP_FLAG'] = (map_snomed_to_gdppr_df['Active_in_Refset']
                                                     .where(map_snomed_to_gdppr_df['Active_in_Refset']
                                                            .isnull(), 1)
                                                     .fillna(0)
                                                     .astype(int))
        

        return map_snomed_to_gdppr_df
    
    
    
    def sort_effective_date_and_assured(self, snomed_gddppr_map_df):
        sort_effective_date_and_assured_df = (snomed_gddppr_map_df
                                              .where((snomed_gddppr_map_df.IS_ASSURED==1) &
                                                     (snomed_gddppr_map_df.MapStatus==1))
                                                .sort_values(['read_code','EffectiveDate'], ascending=False))
        
        sort_effective_date_and_assured_df = (sort_effective_date_and_assured_df
                                              .drop_duplicates(['read_term','ConceptId'], keep='first'))
        
        return sort_effective_date_and_assured_df
                     

Writing readv2_to_snomed_search_gdppr.py


In [3]:
converted_project_codelist_1 = pd.read_excel('codelist_converted.xlsx', sheet_name = 'table_1')
converted_project_codelist_1['read_code'] = converted_project_codelist_1['read_code'].astype(str)

converted_project_codelist_2 = pd.read_excel('codelist_converted.xlsx', sheet_name = 'table_2')
converted_project_codelist_2['read_code'] = converted_project_codelist_2['read_code'].astype(str)

converted_project_codelist_2['read_code'].replace('6951','6951.00',inplace=True)
converted_project_codelist_2['read_code'].replace('1695','1695.00',inplace=True)
converted_project_codelist_2['read_code'].replace('1343','1343.00',inplace=True)
converted_project_codelist_2['read_code'].replace('1345','1345.00',inplace=True)
converted_project_codelist_2['read_code'].replace('1344','1344.00',inplace=True)
converted_project_codelist_2['read_code'].replace('1342','1342.00',inplace=True)
converted_project_codelist_2['read_code'].replace('1347','1347.00',inplace=True)
converted_project_codelist_2['read_code'].replace('1348','1348.00',inplace=True)
converted_project_codelist_2['read_code'].replace('1346','1346.00',inplace=True)
converted_project_codelist_2['read_code'].replace('2263','2263.00',inplace=True)

In [4]:
print(f'Table 1 has {converted_project_codelist_1.read_code.nunique()} unique read_codes')
print(f'Table 2 has {converted_project_codelist_2.read_code.nunique()} unique read_codes')

Table 1 has 168 unique read_codes
Table 2 has 434 unique read_codes


In [5]:
from readv2_to_snomed_search_gdppr import *

Reading in the read to snomed ct mappinging file derrived from nhs digital trud

In [6]:
read_snomed_map = pd.read_csv('rcsctmap2_uk_20200401000001.txt', 
                               sep='\t' ,encoding='unicode_escape')
read_snomed_map.head()

,MapId,ReadCode,TermCode,ConceptId,DescriptionId,IS_ASSURED,EffectiveDate,MapStatus
0,{f9acc33c-2623-11e3-a0b5-00ff3a5bce8f},0....,00,14679004,476525017.0,1,20130925,1
1,{f9acc3d1-2623-11e3-a0b5-00ff3a5bce8f},0....,11,14679004,24938015.0,1,20130925,1
2,{f9acc3ef-2623-11e3-a0b5-00ff3a5bce8f},01...,00,265911003,395138017.0,1,20130925,1
3,{f9acc3fe-2623-11e3-a0b5-00ff3a5bce8f},01...,11,308050009,451353015.0,1,20130925,1
4,{f9acc40a-2623-11e3-a0b5-00ff3a5bce8f},011..,00,158744001,247411016.0,1,20130925,1


Creating a read_term column on the projects read code and term code columns

In [7]:
prepared_read_snomed_map = (projectDataPreparation
                             .create_read_term_column(read_snomed_map, 
                                                       'ReadCode', 
                                                       'TermCode',
                                                        'read_term_map'))
prepared_read_snomed_map

,MapId,ReadCode,TermCode,ConceptId,DescriptionId,IS_ASSURED,EffectiveDate,MapStatus,read_term_map
0,{f9acc33c-2623-11e3-a0b5-00ff3a5bce8f},0....,00,14679004,4.765250e+08,1,20130925,1,0....00
1,{f9acc3d1-2623-11e3-a0b5-00ff3a5bce8f},0....,11,14679004,2.493802e+07,1,20130925,1,0....11
2,{f9acc3ef-2623-11e3-a0b5-00ff3a5bce8f},01...,00,265911003,3.951380e+08,1,20130925,1,01...00
3,{f9acc3fe-2623-11e3-a0b5-00ff3a5bce8f},01...,11,308050009,4.513530e+08,1,20130925,1,01...11
4,{f9acc40a-2623-11e3-a0b5-00ff3a5bce8f},011..,00,158744001,2.474110e+08,1,20130925,1,011..00
...,...,...,...,...,...,...,...,...,...
216866,{31a0528f-7fdf-11e9-b2aa-8c1645000138},Zw05B,00,255203001,3.803560e+08,0,20190526,1,Zw05B00
216867,{c428c54f-c281-11e8-949b-8c1645000138},Zw05B,00,432471000000101,8.926010e+14,0,20190526,0,Zw05B00
216868,{f9c1db68-2623-11e3-a0b5-00ff3a5bce8f},Zw05B,00,432471000000101,8.926010e+14,1,20130925,1,Zw05B00
216869,{c428c54f-c281-11e8-949b-8c1645000138},Zw05B,00,432471000000101,8.926010e+14,0,20180927,1,Zw05B00


In [8]:
def main(project_csv,
         project_codelist_column,
         mapping_file,
         mapping_file_codelist_column,
         mapped_snomed_project_codelist_column,
         mapping_file_gdppr_codelist_column,
         output_prefix
         
        ):

    # map project codes to readv2 map for table 1
    read_to_snomed_obj = readv2TosnomedConversion(project_df = project_csv, 
                                                    project_codelist_column=project_codelist_column, 
                                                    mapping_file = mapping_file, 
                                                    mapping_file_codelist_column = mapping_file_codelist_column)

    project_read_to_snomed_df = read_to_snomed_obj.map_readV2_to_snomed()
    print(project_read_to_snomed_df.nunique())

    project_read_to_snomed_df.to_csv(f'{output_prefix}_read_v2_to_snomedct.csv')
    
    print('For read_v2_to_snomedct mapping')
    conversionChecks.get_codes_not_mapped(project_csv, 
                                          project_codelist_column ,
                                          project_read_to_snomed_df, 
                                          mapping_file_codelist_column,
                                          map_flag_col = 'CONCEPT_MAP_FLAG')

#   hardcoded refactor
    gdppr_codes_clusters = pd.read_csv('GDPPR_Cluster_Refset_1000230_20220429.csv',
                                       encoding='unicode_escape')

    # map project_readv2 to gdppr
    project_read_to_snomed_df['ConceptId'] = project_read_to_snomed_df['ConceptId'].astype('Int64')
    gdppr_codes_clusters['ConceptId'] = gdppr_codes_clusters['ConceptId'].astype('Int64')

    snomedToGdppr_obj = snomedToGdpprLookUp(project_read_to_snomed_df,
                                              mapped_snomed_project_codelist_column = mapped_snomed_project_codelist_column,
                                              mapping_file = gdppr_codes_clusters,
                                              mapping_file_gdppr_codelist_column = mapping_file_gdppr_codelist_column
                                            )
    

    snomed_to_gdppr_df = snomedToGdppr_obj.map_snomed_to_gdppr()
    snomed_to_gdppr_df = snomedToGdppr_obj.sort_effective_date_and_assured(snomed_to_gdppr_df)
    print(snomed_to_gdppr_df.nunique())

    print('For snomedct_to_gdppr mapping')
    conversionChecks.get_codes_not_mapped(project_read_to_snomed_df, 
                                          mapped_snomed_project_codelist_column,
                                          snomed_to_gdppr_df, 
                                          mapping_file_gdppr_codelist_column,
                                          map_flag_col = 'GDPPR_MAP_FLAG')

#     snomed_to_gdppr_df

#     snomed_to_gdppr_df.read_code.value_counts()

#     snomed_to_gdppr_df.GDPPR_MAP_FLAG.value_counts()

    snomed_to_gdppr_df.to_csv(f'{output_prefix}_snomedct_to_gdppr.csv')

In [9]:
main(project_csv=converted_project_codelist_1,
     project_codelist_column ='read_code' ,
     mapping_file = prepared_read_snomed_map,
     mapping_file_codelist_column = 'read_term_map',
     mapped_snomed_project_codelist_column = 'ConceptId',
     mapping_file_gdppr_codelist_column = 'ConceptId',
     output_prefix = 'table_1')

NameError: name 'readv2TosnomedConversion' is not defined

In [ ]:
main(project_csv=converted_project_codelist_2,
     project_codelist_column ='read_code' ,
     
     mapping_file = prepared_read_snomed_map,
     mapping_file_codelist_column = 'read_term_map',
     
     mapped_snomed_project_codelist_column = 'ConceptId',
     mapping_file_gdppr_codelist_column = 'ConceptId',
     
     output_prefix = 'table_2')